## Mounting Google drive to save and retrive the model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Definition of All Required Classes

### Importing Libraries

In [ ]:
import math
%tensorflow_version 2.x
import numpy as np
from collections import deque
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Activation, Dropout, MaxPooling2D, Input, Add, BatchNormalization, LeakyReLU
from tensorflow.keras.models import load_model

### Game class, which has methods and rules to play connect 4

In [ ]:
import numpy as np
from copy import deepcopy

class Game():
    def __init__(self,state=np.zeros((6,7)),player=1):
        self.state = deepcopy(state)
        self.player = player
        #self.top = self.gettop(state)        
        pass
    
    def gettop(self,state):
        top = np.zeros((7,),dtype=np.int32)
        for i in range(7):
            for j in range(6):
                if state[j,i]==0:
                    top[i]=j
                    break
                top[i]=6
        return top
    
    def ValidMoves(self,state,player):
        top=self.gettop(state)
        return top<6    
    
    def Next(self,state,player,action):
        state=deepcopy(state)
        for j in range(6):
            if state[j,action]==0:
                state[j,action]=player
                break
        return state, -player
    
    def play(self,action):
        self.state[self.gettop(self.state)[action],action]=self.player
        self.player*=-1
        pass
    
    def end(self,state,player):
        tok=player
        for i in range(4):
            for j in range(6):
                if state[j][i]==tok and state[j][i+1]==tok and state[j][i+2]==tok and state[j][i+3]==tok:
                    return 1

            for j in range(2):
                if state[j][i]==tok and state[j+1][i+1]==tok and state[j+2][i+2]==tok and state[j+3][i+3]==tok:
                    return 1

            for j in range(3,6):
                if state[j][i]==tok and state[j-1][i+1]==tok and state[j-2][i+2]==tok and state[j-3][i+3]==tok:
                    return 1

        for i in range(7):
            for j in range(3):
                if state[j][i]==tok and state[j+1][i]==tok and state[j+2][i]==tok and state[j+3][i]==tok:
                    return 1

        tok=-player
        for i in range(4):
            for j in range(6):
                if state[j][i]==tok and state[j][i+1]==tok and state[j][i+2]==tok and state[j][i+3]==tok:
                    return -1

            for j in range(2):
                if state[j][i]==tok and state[j+1][i+1]==tok and state[j+2][i+2]==tok and state[j+3][i+3]==tok:
                    return -1

            for j in range(3,6):
                if state[j][i]==tok and state[j-1][i+1]==tok and state[j-2][i+2]==tok and state[j-3][i+3]==tok:
                    return -1   
        for i in range(7):
            for j in range(3):
                if state[j][i]==tok and state[j+1][i]==tok and state[j+2][i]==tok and state[j+3][i]==tok:
                    return -1

        if min(self.gettop(state))==6:
            return 0
        return -2
    
    def fstategen(self,state,player):
        return (state*player)
    
    def stringrep(self,state):
        return state.tobytes()
            
    def PrintBoard(self,state):
        for i in range(5,-1,-1):
            for j in range(7):
                if state[i,j]==1:
                    print("|X", end="")
                elif state[i,j]==-1:
                    print("|O", end="")
                else:
                    print("| ", end="")
            print("|")

### Network class contains neural network and it's required methods

In [ ]:
#%tensorflow_version 2.x
import numpy as np
import math
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Activation, Dropout, MaxPooling2D, Input, Add

class Network():
    def generate_model(self):
        def identity_block(X, f, filters):
            F1, F2 = filters
            
            X_shortcut = X
            
            X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1))(X)
            X = LeakyReLU()(X)
                
            X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
            X = BatchNormalization()(X)
            X = Add()([X_shortcut, X])
            X = LeakyReLU()(X)
            return X
        input = Input(shape=(6,7,2))

        layer = Conv2D(filters = 8, kernel_size = (1, 1), strides = (1,1))(input)
        layer=LeakyReLU()(layer)
        layer=identity_block(layer, 4, [8, 8])
        layer=identity_block(layer, 3, [8, 8])
        layer=MaxPooling2D(pool_size=(3,3),strides=(1,1))(layer)

        layer = Conv2D(filters = 16, kernel_size = (1, 1), strides = (1,1))(layer)
        layer=LeakyReLU()(layer)
        layer=identity_block(layer, 3, [16, 16])
        layer=identity_block(layer, 2, [16, 16])
        layer=MaxPooling2D(pool_size=(2,2),strides=(1,1))(layer)
        layer = Conv2D(filters = 32, kernel_size = (3, 3), strides = (1,1))(layer)
        layer=LeakyReLU()(layer)

        layer=Flatten()(layer)

        #--------VALUE NETWORK---------
        valuehead=Dense(32)(layer)
        layer=LeakyReLU()(layer)
        valuehead = Dropout(0.4)(valuehead)

        valuehead=Dense(8)(valuehead)
        valuehead=LeakyReLU()(valuehead)
        valuehead = Dropout(0.2)(valuehead)
        
        valuehead=Dense(4)(valuehead)
        valuehead=LeakyReLU()(valuehead)

        valuehead=Dense(1, activation='tanh', name="valuehead")(valuehead)


        #--------POLICY NETWORK---------
        policyhead=Dense(56)(layer)
        policyhead=LeakyReLU()(policyhead)
        policyhead = Dropout(0.3)(policyhead)

        policyhead=Dense(42)(policyhead)
        policyhead=LeakyReLU()(policyhead)
        policyhead = Dropout(0.1)(policyhead)

        policyhead=Dense(14)(policyhead)
        policyhead=LeakyReLU()(policyhead)

        policyhead=Dense(7, activation='softmax',name="policyhead")(policyhead)

        model = Model(inputs=input, outputs=[valuehead ,policyhead])
        opt = tf.keras.optimizers.SGD(momentum=0.1)
        loss1 = tf.keras.losses.MeanSquaredError()
        loss2 = self.customLoss
        losses = {
            "valuehead": loss1,
            "policyhead": loss2,
        }
        model.compile(loss=losses, optimizer=opt)
        return model

    def __init__(self, name=None):
        if name==None:
            self.model=self.generate_model()
            self.champion=self.generate_model()
        else:
            self.model = load_model(name, custom_objects={self.customLoss.__name__: self.customLoss})
            self.champion = load_model(name, custom_objects={self.customLoss.__name__: self.customLoss})
            
    def champ_predict(self, state, p):
        st=np.reshape(state, (1,state.shape[0],state.shape[1],-1) )
        inp1=(st*p>0).astype(np.float32)
        inp2=(st*p<0).astype(np.float32)
        inp=np.concatenate( (inp1,inp2 ), axis=3)
        return self.champion.predict(inp)

    def predict(self, state, p):
        st=np.reshape(state, (1,state.shape[0],state.shape[1],-1) )
        inp1=(st*p>0).astype(np.float32)
        inp2=(st*p<0).astype(np.float32)
        inp=np.concatenate( (inp1,inp2 ), axis=3)
        return self.model.predict(inp)
    
    def still_champ(self,name="check"):
        self.champion.save(name)
        tf.keras.backend.clear_session()
        self.model = load_model(name, custom_objects={self.customLoss.__name__: self.customLoss})
        self.champion = load_model(name, custom_objects={self.customLoss.__name__: self.customLoss})
    
    def new_champ(self,name="check"):
        self.model.save(name)
        tf.keras.backend.clear_session()        
        self.model = load_model(name, custom_objects={self.customLoss.__name__: self.customLoss})
        self.champion = load_model(name, custom_objects={self.customLoss.__name__: self.customLoss})
    
    @tf.function
    def customLoss(self,y_true, y_pred):
        #print(y_true)
        #print(y_pred)
        loss =tf.reduce_mean(tf.reduce_sum(-tf.math.multiply(y_true, tf.math.log(y_pred)), axis=1, keepdims=True) )
        return loss

    def preprocess(self,X,Y1,Y2):
        X=np.array(X)
        Y1=np.array(Y1).astype(np.float32)
        Y2=np.array(Y2).astype(np.float32)
        X=np.reshape(X,(X.shape[0],X.shape[1],X.shape[2],-1))
        inp1=(X>0).astype(np.float32)
        inp2=(X<0).astype(np.float32)
        X=np.concatenate( (inp1,inp2 ), axis=3)
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)
        X=X[indices]
        Y1 = Y1[indices]
        Y2 = Y2[indices]
        return X,Y1,Y2

    def train(self, X,Y1,Y2,Epochs=3, Batch=64):        
        X,Y1,Y2=self.preprocess(X, Y1, Y2)
        Y= {"valuehead": Y1,
            "policyhead": Y2 }
        #print(X.shape)
        #print(X[5,:,:,0])
        #print(X[5,:,:,1])
        #print(Y)
        self.model.fit(X, Y, batch_size=Batch, epochs=Epochs)
        pass

### Monte Carlo tree search, Used to find best move.

In [ ]:
class Mcts():
    def __init__(self,game,nn,cput,num):
        self.game = game
        self.nn = nn
        self.cput = cput
        self.num = num
        self.Qsa = {}
        self.Nsa = {}
        self.Ns = {}
        self.Ps = {}
        self.Es = {}
        self.Vs = {}
    
    def search(self,fstate):
        s = fstate.tobytes()
        
        if s not in self.Es:
            self.Es[s] = self.game.end(fstate,1)
        if self.Es[s]!=-2:
            return -self.Es[s]
        
        if s not in self.Ps:
            [v,tempP] = self.nn.predict(fstate,1)
            v = v[0][0]
            self.Ps[s] = tempP[0]
            valids = self.game.ValidMoves(fstate,1)
            self.Ps[s] = self.Ps[s]*valids
            sPs = np.sum(self.Ps[s])
            if sPs > 0:
                self.Ps[s] = self.Ps[s]/sPs
            else:
                self.Ps[s] = self.Ps[s] + valids
                self.Ps[s] = self.Ps[s]/np.sum(self.Ps[s])
            
            self.Vs[s] = valids
            self.Ns[s] = 0
            return -v
        
        valids = self.Vs[s]
        best = -float('inf')
        best_act = -1
        for a in range(7):
            if valids[a]:
                if (s,a) in self.Qsa:
                    PUCT = self.Qsa[(s,a)] + self.cput * self.Ps[s][a] * math.sqrt(self.Ns[s]) / (1+self.Nsa[(s,a)])
                else:
                    PUCT = self.cput * self.Ps[s][a] * math.sqrt(self.Ns[s])
                
                if PUCT > best:
                    best = PUCT
                    best_act = a
        
        a = best_act
        next_state,next_player = self.game.Next(fstate,1,best_act)
        next_fstate = self.game.fstategen(next_state,next_player)
        
        v = self.search(deepcopy(next_fstate))
        
        if (s, a) in self.Qsa:
            self.Qsa[(s,a)]=(self.Nsa[(s,a)]*self.Qsa[(s,a)]+v)/(self.Nsa[(s,a)]+1)
            self.Nsa[(s,a)]+=1

        else:
            self.Qsa[(s,a)]=v
            self.Nsa[(s,a)]=1

        self.Ns[s]+=1
        return -v
    
    def getprobs(self,fstate):
        nu=self.num
        s = fstate.tobytes()
        if s in self.Ns:
            nu=nu-self.Ns[s]-1
        for i in range(nu):
            self.search(deepcopy(fstate))
        
        probs = [self.Nsa[(s, a)] if (s, a) in self.Nsa else 0 for a in range(7)]
        #print(probs)
        probSum = float(sum(probs))
        if probSum==0:
            return probs
        else:
            probs = [x/probSum for x in probs]
        return probs
    
    def getAction(self,fstate):
        probs=self.getprobs(fstate)
        best=-float('inf')
        best_act=-1
        for a in range(len(probs)):
            if probs[a]>best:
                best=probs[a]
                best_act=a
        return best_act

### selfplay class to Train the agent

In [ ]:
import numpy as np
import random

class selfplay():
    def __init__(self):
        pass
        
    def play(self,toprint,mu):
        examples = []
        self.game.state=np.zeros((6,7))
        self.game.player=1
        self.mcts=Mcts(self.game,self.nn,self.cput,self.num)
        while True:
            #self.mcts=Mcts(self.game,self.nn,self.cput,self.num)
            
            if toprint:
                print("""
                ##########################################
                ##########################################
                """)
                self.game.PrintBoard(self.game.state)
                print("")
            
            fstate = self.game.fstategen(self.game.state,self.game.player)
            pi = np.array(self.mcts.getprobs(deepcopy(fstate)))
            pro=np.exp(pi*mu)*self.game.ValidMoves(fstate,1)
            pro=pro/np.sum(pro)
            if toprint:
                print(pi)
                print(pro)
            best_act=np.random.choice(7,p=pro)
            #best_act=np.random.choice(7,p=pi)
            examples.append([deepcopy(self.game.state),self.game.player,deepcopy(pi)])

            self.game.play(best_act)
            r = self.game.end(self.game.state,self.game.player)
            if r!=-2:
                if toprint:
                    self.game.PrintBoard(self.game.state)
                    print("""
                    ---------------------------------------------------------------------
                    ---------------------------------------------------------------------
                    """)
                return [[x[0]*x[1] for x in examples], [r*self.game.player*x[1] for x in examples], [x[2] for x in examples]]

            #examples.append([deepcopy(self.game.state),self.game.player,deepcopy(pi)])
    
    def learn(self,game,nn,cput,num,no_of_games, toprint=False, mu=30):
        self.game=game
        self.nn=nn
        self.cput=cput
        self.num=num
        self.no_of_games=no_of_games
        self.mcts=Mcts(self.game,self.nn,self.cput,self.num)
        X, Y1, Y2 = [],[],[]
        for _ in range(self.no_of_games):
            print("no_of_games",_)
            x,y1,y2=self.play(toprint,mu)
            X+=x
            Y1+=y1
            Y2+=y2
            # for i in range(len(X)):
                # print(X[i])
                # print(Y1[i])
                # print(Y2[i])
            
        self.nn.train(X,Y1,Y2,10)

# Training phase

In [ ]:
game = Game()
nn = Network()
p = selfplay()

In [ ]:
p.learn(game,nn,1,20,10,False)
p.learn(game,nn,1,20,10,False)
#p.learn(game,nn,1,20,10,False)
#p.learn(game,nn,1,20,10,False)

no_of_games 0
no_of_games 1
no_of_games 2
no_of_games 3
no_of_games 4
no_of_games 5
no_of_games 6
no_of_games 7
no_of_games 8
no_of_games 9
Epoch 1/10
3/3 [==============================] - 0s 135ms/step - loss: 3.2549 - valuehead_loss: 1.2834 - policyhead_loss: 1.9716
Epoch 2/10
3/3 [==============================] - 0s 8ms/step - loss: 3.0494 - valuehead_loss: 1.0952 - policyhead_loss: 1.9541
Epoch 3/10
3/3 [==============================] - 0s 8ms/step - loss: 2.9721 - valuehead_loss: 1.0094 - policyhead_loss: 1.9626
Epoch 4/10
3/3 [==============================] - 0s 8ms/step - loss: 2.9149 - valuehead_loss: 0.9495 - policyhead_loss: 1.9654
Epoch 5/10
3/3 [==============================] - 0s 8ms/step - loss: 2.8648 - valuehead_loss: 0.9124 - policyhead_loss: 1.9524
Epoch 6/10
3/3 [==============================] - 0s 8ms/step - loss: 2.8253 - valuehead_loss: 0.8772 - policyhead_loss: 1.9481
Epoch 7/10
3/3 [==============================] - 0s 8ms/step - loss: 2.7532 - valuehead_l

In [ ]:
nn.new_champ()

# The champion vs contendor to evaluate the agent's skills (Incomplete)

In [ ]:
class MctsChamp():
    def __init__(self,game,nn,cput,num):
        self.game = game
        self.nn = nn
        self.cput = cput
        self.num = num
        self.Qsa = {}
        self.Nsa = {}
        self.Ns = {}
        self.Ps = {}
        self.Es = {}
        self.Vs = {}
    
    def search(self,fstate):
        s = fstate.tobytes()
        
        if s not in self.Es:
            self.Es[s] = self.game.end(fstate,1)
        if self.Es[s]!=-2:
            return -self.Es[s]
        
        if s not in self.Ps:
            [v,tempP] = self.nn.champ_predict(fstate,1)
            v = v[0][0]
            self.Ps[s] = tempP[0]
            valids = self.game.ValidMoves(fstate,1)
            self.Ps[s] = self.Ps[s]*valids
            sPs = np.sum(self.Ps[s])
            if sPs > 0:
                self.Ps[s] = self.Ps[s]/sPs
            else:
                self.Ps[s] = self.Ps[s] + valids
                self.Ps[s] = self.Ps[s]/np.sum(self.Ps[s])
            
            self.Vs[s] = valids
            self.Ns[s] = 0
            return -v
        
        valids = self.Vs[s]
        best = -float('inf')
        best_act = -1
        for a in range(7):
            if valids[a]:
                if (s,a) in self.Qsa:
                    PUCT = self.Qsa[(s,a)] + self.cput * self.Ps[s][a] * math.sqrt(self.Ns[s]) / (1+self.Nsa[(s,a)])
                else:
                    PUCT = self.cput * self.Ps[s][a] * math.sqrt(self.Ns[s])
                
                if PUCT > best:
                    best = PUCT
                    best_act = a
        
        a = best_act
        next_state,next_player = self.game.Next(fstate,1,best_act)
        next_fstate = self.game.fstategen(next_state,next_player)
        
        v = self.search(deepcopy(next_fstate))
        
        if (s, a) in self.Qsa:
            self.Qsa[(s,a)]=(self.Nsa[(s,a)]*self.Qsa[(s,a)]+v)/(self.Nsa[(s,a)]+1)
            self.Nsa[(s,a)]+=1

        else:
            self.Qsa[(s,a)]=v
            self.Nsa[(s,a)]=1

        self.Ns[s]+=1
        return -v
    
    def getprobs(self,fstate):
        nu=self.num
        s = fstate.tobytes()
        if s in self.Ns:
            nu=nu-self.Ns[s]-1
        for i in range(nu):
            self.search(deepcopy(fstate))
        
        probs = [self.Nsa[(s, a)] if (s, a) in self.Nsa else 0 for a in range(7)]
        #print(probs)
        probSum = float(sum(probs))
        if probSum==0:
            return probs
        else:
            probs = [x/probSum for x in probs]
        return probs
    
    def getAction(self,fstate):
        probs=self.getprobs(fstate)
        best=-float('inf')
        best_act=-1
        for a in range(len(probs)):
            if probs[a]>best:
                best=probs[a]
                best_act=a
        return best_act

In [ ]:
def championship(game,para=[]):
    result=[]
    turn=0
    for turn in range(2):
        player1=Mcts(*para)
        player2=MctsChamp(*para)
        game.state=np.zeros((6,7))
        game.player=1
        while True:
            #if toprint:
            print("""
            ##########################################
            """)
            game.PrintBoard(game.state)
            # print("""
            # ##########################################
            # ##########################################
            # """)
            fstate = game.fstategen(game.state,game.player)
            if turn%2==0:
                print("contendor")
                best_act=player1.getAction(deepcopy(fstate))
            else:
                print("champ")
                best_act=player2.getAction(deepcopy(fstate))
            turn+=1
            game.play(best_act)
            r = game.end(game.state,1)
            if r!=-2:
                game.PrintBoard(game.state)
                print("""
                ---------------------------------------------------------------------
                ---------------------------------------------------------------------
                """)
                print(r)
                result.append((r*((-1)**turn)))
                break

        player1,player2=player2,player1
    print(result)
    return result

In [ ]:
class playWith():
    def __init__(self):
        pass
        
    def play(self,game,nn,cput,num,pos):
        self.game=deepcopy(game)
        self.nn=nn
        self.cput=cput
        self.num=num
        self.game.state=np.zeros((6,7))
        self.game.player=1
        self.mcts=Mcts(self.game,self.nn,self.cput,self.num)
        examples = []
        self.game.PrintBoard(self.game.state)
        print("""
        ##########################################
        ##########################################
        """)
            
        while True:            

            if self.game.player==(-1)*pos:
                print("AI's Turn")
                print("Selecting Action........")
                print(" 0 1 2 3 4 5 6 ")            
                fstate = self.game.fstategen(self.game.state,self.game.player)
                pi = self.mcts.getprobs(deepcopy(fstate))
                best_act = self.mcts.getAction(deepcopy(fstate))
                examples.append([deepcopy(self.game.state),self.game.player,deepcopy(pi)])
                self.game.play(best_act)
                self.game.PrintBoard(self.game.state)
                print("""
                ##########################################
                ##########################################
                """)
            
            else:
                print("Your Turn")
                best_act = int(input("Your Action:"))
                print(" 0 1 2 3 4 5 6 ")
                self.game.play(best_act)
                self.game.PrintBoard(self.game.state)
                print("""
                ##########################################
                ##########################################
                """)
                
            r = self.game.end(self.game.state,self.game.player)
            if r!=-2:
                print("Game Ended!!!")
                if self.game.player==pos:
                    if r==1:
                        print("You Won!!!")
                    if r==-1:
                        print("You Lost!!!")
                    if r==0:
                        print("It is a draw!!!")
                else:
                    if r==1:
                        print("You Lost!!!")
                    if r==-1:
                        print("You Won!!!")
                    if r==0:
                        print("It is a draw!!!")
                return [[x[0]*x[1] for x in examples], [r*self.game.player*x[1] for x in examples], [x[2] for x in examples]]
                #return 1, 2, 3